In [11]:
from __future__ import print_function

import math
import time

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from numpy import array
import pandas as pd
#from sklearn.metrics import accuracy_score,classification_report, confusion_matrix , mean_squared_error, recall_score,precision_score,f1_score
#IMPORT KERAS LIBRARIES (Using Tensorflow backend)
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
import keras.layers
from keras import metrics
import keras_metrics as km
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format


#Importing the data as panda dataframe

intrusion_data_df = pd.read_csv("C:/datasets/NSLKDD/Clean/KDDTrain.csv", sep=",", header=None)
intrusion_data_df_test = pd.read_csv("C:/datasets/NSLKDD/Clean/KDDTest.csv", sep=",", header=None)

intrusion_data_df.columns = ["duration","protocol_type","service","flag","src_bytes","dst_bytes",
                             "land","wrong_fragment","urgent","hot","num_failed_logins","logged_in",
                             "num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
                             "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count",
                             "serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate",
                             "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
                             "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
                             "dst_host_rerror_rate","dst_host_srv_rerror_rate","class"]
intrusion_data_df_test.columns = ["duration","protocol_type","service","flag","src_bytes","dst_bytes",
                             "land","wrong_fragment","urgent","hot","num_failed_logins","logged_in",
                             "num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells",
                             "num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count",
                             "serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate",
                             "srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
                             "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
                             "dst_host_rerror_rate","dst_host_srv_rerror_rate","class"]

#intrusion_data_df_test = intrusion_data_df_test.fillna(0)


attack_types_dict = {
        'back': 'dos',
        'buffer_overflow': 'u2r',
        'ftp_write': 'r2l',
        'guess_passwd': 'r2l',
        'imap': 'r2l',
        'ipsweep': 'probe',
        'land': 'dos',
        'loadmodule': 'u2r',
        'multihop': 'r2l',
        'neptune': 'dos',
        'nmap': 'probe',
        'perl': 'u2r',
        'phf':'r2l',
        'pod':'dos',
        'portsweep': 'probe',
        'rootkit': 'u2r',
        'satan': 'probe',
        'smurf': 'dos',
        'spy':'r2l',
        'teardrop': 'dos',
        'warezclient': 'r2l',
        'warezmaster':'r2l',
        'normal': 'normal'
}

class_mapping = {
    "r2l" : 1,
    "u2r" : 2,
    "probe" : 3,
    "dos" : 4,
    "normal" : 0
}

#step 1 mapping
intrusion_data_df["class"] = intrusion_data_df["class"].map(attack_types_dict)
intrusion_data_df_test["class"] = intrusion_data_df_test["class"].map(attack_types_dict)


#setp 2 mapping
intrusion_data_df["class"] = intrusion_data_df["class"].map(class_mapping)
intrusion_data_df_test["class"] = intrusion_data_df_test["class"].map(class_mapping)


#Random Permutations
intrusion_data_df = intrusion_data_df.reindex(np.random.permutation(intrusion_data_df.index))
intrusion_data_df_test = intrusion_data_df_test.reindex(np.random.permutation(intrusion_data_df_test.index))


###### Convert Categorical Variables #######

convert_protocol_type = {
    "tcp" : 1,
    "udp" : 2,
    "icmp" : 3
}

convert_flag = {
    "SF": 1,       
    "REJ": 2,        
    "S0": 3,       
    "RSTO" :4,       
    "RSTR": 5,       
    "S3" : 6 ,      
    "SH" : 7,         
    "S1":  8,        
    "S2": 9,         
    "OTH":10,           
    "RSTOS0": 11
}

convert_service = {
    "http":            1,
    "private":         2,
    "domain_u":        3,
    "smtp":            4,
    "ftp_data":        5,
    "eco_i":           6,
    "other":           7,
    "ecr_i":           8,
    "telnet":          9,
    "finger":          10,
    "ftp":             11,
    "auth":            12,
    "Z39_50":          13,
    "uucp":            14,
    "courier":         15,
    "bgp":             16,
    "whois":           17,
    "uucp_path":       18,
    "iso_tsap":        19,
    "time":            20,
    "imap4":           21,
    "nnsp":            22,
    "vmnet" :          23,
    "urp_i":           24,
    "domain":          25,
    "ctf" :            26,
    "csnet_ns":        27,
    "supdup":          28,
    "discard" :        29,
    "http_443":        30,
    "daytime":         31,
    "gopher":          32,
    "efs"  :           33,
    "systat" :         34,
    "link"  :          35,
    "exec":            36,
    "hostnames":       37,
    "name"  :          38,
    "mtp" :            39,
    "echo":            40,
    "klogin" :         41,
    "login":           42,
    "ldap":            43,
    "netbios_dgm":     44,
    "sunrpc" :         45,
    "netbios_ssn":     46,
    "netstat" :        47,
    "netbios_ns":      48,
    "ssh" :            49,
    "kshell" :         50,
    "nntp" :           51,
    "pop_3" :          52,
    "sql_net":         53,
    "IRC" :            54,
    "ntp_u":           55,
    "rje":             56,
    "remote_job":      57,
    "pop_2":           58,
    "X11" :            59,
    "printer":         60,
    "shell" :          61,
    "urh_i" :          62,
    "tim_i" :          63,
    "red_i" :          64,
    "pm_dump":         65,
    "tftp_u" :         66,
    "http_8001" :      67,
    "aol":             68,
    "harvest" :        69,
    "http_2784":       70
}


intrusion_data_df["protocol_type"] = intrusion_data_df["protocol_type"].map(convert_protocol_type)
intrusion_data_df_test["protocol_type"] = intrusion_data_df_test["protocol_type"].map(convert_protocol_type)

intrusion_data_df["flag"] = intrusion_data_df["flag"].map(convert_flag)
intrusion_data_df_test["flag"] = intrusion_data_df_test["flag"].map(convert_flag)

intrusion_data_df["service"] = intrusion_data_df["service"].map(convert_service)
intrusion_data_df_test["service"] = intrusion_data_df_test["service"].map(convert_service)

###########################################

#intrusion_data_df.head()

#Normalization functions
def log_normalize(series):
  return series.apply(lambda x:math.log(x+1.0))

# x between 0 and 1

def normalize_0_1(series):
    return series.apply(lambda x:(x-min(x))/(max(x)-min(x)))


#Function to Process the features / inputs
def preprocess_inputs(intrusion_data_df):
    '''The Features selected here were 
    generated by another feature reducation algorithm.
    - These are the most relevant ones
    '''
    processed_inputs = pd.DataFrame()
    processed_inputs["duration"] = log_normalize(intrusion_data_df["duration"])
    processed_inputs["protocol_type"] = log_normalize(intrusion_data_df["protocol_type"])
    processed_inputs["flag"] = log_normalize(intrusion_data_df["flag"])
    processed_inputs["service"] = log_normalize(intrusion_data_df["service"])
    processed_inputs["src_bytes"] = log_normalize(intrusion_data_df["src_bytes"])
    processed_inputs["num_failed_logins"] = log_normalize(intrusion_data_df["num_failed_logins"])
    processed_inputs["is_guest_login"] = log_normalize(intrusion_data_df["is_guest_login"])
    processed_inputs["srv_count"] = log_normalize(intrusion_data_df["srv_count"])
    processed_inputs["serror_rate"] = log_normalize(intrusion_data_df["serror_rate"])
    processed_inputs["srv_rerror_rate"] = log_normalize(intrusion_data_df["srv_rerror_rate"])
    processed_inputs["srv_diff_host_rate"] = log_normalize(intrusion_data_df["srv_diff_host_rate"])
    processed_inputs["dst_host_count"] = log_normalize(intrusion_data_df["dst_host_count"])
    processed_inputs["dst_host_srv_count"] = log_normalize(intrusion_data_df["dst_host_srv_count"])
    processed_inputs["dst_host_same_srv_rate"] = log_normalize(intrusion_data_df["dst_host_same_srv_rate"])
    processed_inputs["dst_host_diff_srv_rate"] = log_normalize(intrusion_data_df["dst_host_diff_srv_rate"])
    processed_inputs["dst_host_srv_diff_host_rate"] = log_normalize(intrusion_data_df["dst_host_srv_diff_host_rate"])
    processed_inputs["dst_host_serror_rate"] = log_normalize(intrusion_data_df["dst_host_serror_rate"])
    processed_inputs["dst_host_srv_serror_rate"] = log_normalize(intrusion_data_df["dst_host_srv_serror_rate"])

    return processed_inputs
    
#Function to Process Targets
def preprocess_targets(intrusion_data_df):
    output_targets = pd.DataFrame()
    output_targets["class"] = intrusion_data_df["class"]
    return output_targets
 
 
#Select the first  100778 for training
training_inputs = preprocess_inputs(intrusion_data_df.head(94480))
training_targets =preprocess_targets(intrusion_data_df.head(94480))
encode_training_targets = np_utils.to_categorical(training_targets)


#Select data for 25165 validation
validation_inputs = preprocess_inputs(intrusion_data_df.tail(31493))
validation_targets =preprocess_targets(intrusion_data_df.tail(31493))
encode_validation_targets = np_utils.to_categorical(validation_targets)

#Select the last  for testing
test_inputs = preprocess_inputs(intrusion_data_df_test)
test_targets = preprocess_targets(intrusion_data_df_test)


#Replacing att the inf and -inf to nan;  replace nan with 0s
true_test_targets_0 = test_targets.replace([np.inf, -np.inf], np.nan).fillna(0)
true_test_targets = pd.DataFrame(data = true_test_targets_0, dtype=np.int8)
encode_test_targets = np_utils.to_categorical(true_test_targets)

#############################################
#DLSTM Data Transformation

dlstm_training_inputs = preprocess_inputs(intrusion_data_df.head(94480))
dlstm_training_inputs = array(dlstm_training_inputs)
dlstm_training_inputs = dlstm_training_inputs.reshape(94480,1,18)

dlstm_training_targets =preprocess_targets(intrusion_data_df.head(94480))
dlstm_encode_training_targets = np_utils.to_categorical(training_targets)


#Select data for 25165 validation
dlstm_validation_inputs = preprocess_inputs(intrusion_data_df.tail(31493))
dlstm_validation_inputs = array(dlstm_validation_inputs)
dlstm_validation_inputs = dlstm_validation_inputs.reshape(31493,1,18)

dlstm_validation_targets =preprocess_targets(intrusion_data_df.tail(31493))
dlstm_encode_validation_targets = np_utils.to_categorical(dlstm_validation_targets)

#Select the last  for testing
dlstm_test_inputs = preprocess_inputs(intrusion_data_df_test)
dlstm_test_inputs = array(dlstm_test_inputs)
dlstm_test_inputs = dlstm_test_inputs.reshape(22543,1,18)

dlstm_test_targets = preprocess_targets(intrusion_data_df_test)


#Replacing att the inf and -inf to nan;  replace nan with 0s
dlstm_true_test_targets_0 = dlstm_test_targets.replace([np.inf, -np.inf], np.nan).fillna(0)
dlstm_true_test_targets = pd.DataFrame(data = dlstm_true_test_targets_0, dtype=np.int8)
dlstm_encode_test_targets = np_utils.to_categorical(dlstm_true_test_targets)


#Creating the model DLSTM

dlstm_model = Sequential()
dlstm_model.add(LSTM(50,return_sequences=True, input_shape=(1,18)))
dlstm_model.add(LSTM(50,return_sequences=True))
dlstm_model.add(LSTM(50))
dlstm_model.add(Dense(10))
dlstm_model.add(Dense(5, activation='softmax'))

# Compiling model
dlstm_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy',km.binary_precision(),km.binary_recall()]) #categorical_crossentropy mean_squared_error optimizer='adam'

#start time of training
start = time.time()
# Training a model / Fit Model
dlstm_neural_net_model= dlstm_model.fit(dlstm_training_inputs, dlstm_encode_training_targets,
                                        validation_data=(dlstm_validation_inputs,dlstm_encode_validation_targets), epochs=100, batch_size=500, verbose=0)

#End time of training
end = time.time()

#Training Time
delay = end - start

# evaluate the model


#validation Scores
dlstm_validation_scores = dlstm_model.evaluate(dlstm_validation_inputs, dlstm_encode_validation_targets)

#Test Scores
dlstm_test_scores = dlstm_model.evaluate(dlstm_test_inputs, dlstm_encode_test_targets)

#Compute f1_score - F Measure  
precision = dlstm_validation_scores[2] * 100
recall = dlstm_validation_scores[3] * 100

f1score = (2 * precision * recall)/(precision + recall)

print("\n DLSTM Validation Accuracy: %.2f%%" % (dlstm_validation_scores[1]*100))
print("\n DLSTM Test Accuracy: %.2f%%" % (dlstm_test_scores[1]*100))
print("\n DLSTM Validation Precision: %.2f%%" % (precision))
print("\n DLSTM Validation Recall: %.2f%%" % (recall))
print("\n DLSTM Validation F1- Score: %.2f%%" % (f1score))
print("Training Time "+ str(delay))


22543/22543 [==============================] - 2s 96us/step

 DLSTM Validation Accuracy: 99.49%

 DLSTM Test Accuracy: 86.69%

 DLSTM Validation Precision: 99.55%

 DLSTM Validation Recall: 99.86%

 DLSTM Validation F1- Score: 99.70%
Training Time 935.5980608463287
